In [1]:
from ray.rllib.models.torch.torch_distributions import TorchCategorical, TorchDeterministic, TorchDiagGaussian, TorchMultiDistribution
import numpy as np
import torch
import os 


In [2]:
A = torch.tensor(np.ones((5,6)))


In [3]:
TorchCategorical.from_logits(torch.tensor(np.zeros((5, 5, 100)))).sample()

tensor([[79, 60, 96, 40, 45],
        [ 6, 33, 73, 80, 45],
        [58, 96, 72, 12, 54],
        [83, 17, 10, 20, 15],
        [72, 82, 66, 18, 18]])

In [4]:
TorchDiagGaussian.from_logits(A).sample().shape

torch.Size([5, 3])

In [6]:
TorchMultiDistribution({
    'action1': TorchDeterministic(A),
    'action2': TorchDeterministic(A)
}).sample()

{'action1': tensor([[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]], dtype=torch.float64),
 'action2': tensor([[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]], dtype=torch.float64)}

In [69]:
%load_ext autoreload
%autoreload 2
from train import RayWrapper
from policy.net import Net
import tree
from luxai_s2.env import LuxAI_S2
from dataclasses import asdict
from impl_config import EnvParam
from lux.config import EnvConfig
from lux.kit import obs_to_game_state
from helpers.get_valid_actions import get_valid_actions
from policy.early_setup_player import EarlySetupPlayer

env_cfg = {
        **asdict(EnvConfig()),
                "MIN_FACTORIES": EnvParam.MIN_FACTORIES,
                "MAX_FACTORIES": EnvParam.MAX_FACTORIES,
                "verbose": 0,
                "collect_stats": True,
                "FACTORY_WATER_CONSUMPTION": 0,
                "max_episode_length": 1000,
                "BIDDING_SYSTEM": True
}

wrapper = RayWrapper(env_cfg)
obs = wrapper.reset()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
def get_mock_obs():
    sample_obs = RayWrapper.get_observation_space().sample()['player_0']
    sample_obs = tree.map_structure(lambda x: torch.tensor(x[None]), sample_obs)
    return sample_obs

def get_real_obs():
    sample_obs = obs['my_obs']
    sample_obs = {
        'global_feature': torch.tensor(sample_obs.global_features[None]),
        'map_feature': torch.tensor(sample_obs.map_feature[None]),
        'action_feature': tree.map_structure(lambda x: torch.tensor(x[None]), sample_obs.action_feature)
    }
    return sample_obs

mock_observations = False
sample_obs = get_mock_obs() if mock_observations else get_real_obs()

valid_actions = obs['valid_actions']
valid_actions = tree.map_structure(lambda x: torch.tensor(x[None]), valid_actions)


In [76]:
net = Net()

output = logp, critic_value, action, entropy = net(**sample_obs, va=valid_actions)
output

(tensor([0.]),
 tensor([-0.0002], grad_fn=<MeanBackward1>),
 {'factory_act': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]]),
  'unit_act': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
  
           [[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
  
           [[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0